In [9]:
import pandas as pd
from pathlib import Path
import hvplot.pandas
import numpy as np
# Imports for Alapaca SDK
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

# Alpaca SDK Section

In [10]:
load_dotenv()

True

In [11]:
# define Alpaca Keys
alpaca_api_key = os.getenv('APCA_API_KEY_ID')
alpaca_secret_key = os.getenv('APCA_API_SECRET_KEY')
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

str

str

In [16]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    'https://data.alpaca.markets',
    api_version="v2")

In [19]:
amd_df = alpaca.get_bars("AMD", TimeFrame.Day, "2018-01-01", "2022-04-30", adjustment='raw').df


# Data from CSV section

In [20]:
# Import BTC csv as dataframe and preview first five rows.
btc_df = pd.read_csv(Path('BTC-USD.csv'),
                    index_col="Date",
                    parse_dates=True,
                    infer_datetime_format=True)
btc_df = btc_df.round(decimals=2)
btc_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-28,11475.3,12040.3,11475.30,11786.30,11786.30,8350360064
2018-01-29,11755.5,11875.6,11179.20,11296.40,11296.40,7107359744
2018-01-30,11306.8,11307.2,10036.20,10106.30,10106.30,8637859840
2018-01-31,10108.2,10381.6,9777.42,10221.10,10221.10,8041160192
2018-02-01,10237.3,10288.8,8812.28,9170.54,9170.54,9959400448


In [21]:
# Drop columns from BTC DF
btc_df = btc_df.drop(columns=['Open','High','Low','Adj Close', 'Volume'])
btc_df.head()

,Close
Date,
2018-01-28,11786.30
2018-01-29,11296.40
2018-01-30,10106.30
2018-01-31,10221.10
2018-02-01,9170.54


In [22]:
# Plot price action
btc_df.hvplot(x="Date",y="Close")

:Curve   [Date]   (Close)

In [23]:
# Add the trade_type column to track buys and sells
btc_df['Trade_Type'] = np.nan


# Moving Average Section

In [24]:
# Moving averages
short_window = 50
long_window = 100
ex_long_window = 200

btc_df['SMA50'] = btc_df['Close'].rolling(window=short_window).mean()
btc_df['SMA100'] = btc_df['Close'].rolling(window=long_window).mean()
btc_df['SMA200'] = btc_df['Close'].rolling(window=ex_long_window).mean()

# Create a column to hold the trading signal
btc_df["Signal"] = 0.0

In [25]:
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA100) greater than the long-window (SMA200)
# and 0 is when the condition is not met
btc_df["Signal"][long_window:] = np.where(
    btc_df["SMA100"][long_window:] > btc_df["SMA200"][long_window:], 1.0, 0.0)

# Review the DataFrame
btc_df.loc["2019-05-01":"2019-06-17"]

,Close,Trade_Type,SMA50,SMA100,SMA200,Signal
Date,,,,,,
2019-05-01,5402.70,NaN,4746.8002,4233.8771,4481.84480,0.0
2019-05-02,5505.28,NaN,4778.7714,4252.8841,4477.91655,0.0
2019-05-03,5768.29,NaN,4815.6498,4274.7158,4473.77530,0.0
2019-05-04,5831.17,NaN,4853.0550,4297.0188,4469.95060,0.0
2019-05-05,5795.71,NaN,4887.9946,4318.9782,4466.20700,0.0
2019-05-06,5746.81,NaN,4922.4262,4340.4217,4462.55750,0.0
2019-05-07,5829.50,NaN,4958.3660,4362.8770,4459.37795,0.0
2019-05-08,5982.46,NaN,4996.5914,4387.9971,4456.84430,0.0
2019-05-09,6174.53,NaN,5038.3324,4415.2612,4455.30520,0.0


In [26]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
btc_df["Entry/Exit"] = btc_df["Signal"].diff()

# Review the DataFrame
btc_df.loc["2019-11-01":"2019-12-17"]

,Close,Trade_Type,SMA50,SMA100,SMA200,Signal,Entry/Exit
Date,,,,,,,
2019-11-01,9261.10,NaN,8804.0676,9624.1851,9070.20360,1.0,0.0
2019-11-02,9324.72,NaN,8783.3510,9618.3139,9090.64940,1.0,0.0
2019-11-03,9235.35,NaN,8760.8970,9611.9644,9110.56645,1.0,0.0
2019-11-04,9412.61,NaN,8742.1950,9611.3137,9131.13755,1.0,0.0
2019-11-05,9342.53,NaN,8723.5098,9609.2104,9151.33115,1.0,0.0
2019-11-06,9360.88,NaN,8705.9020,9607.6277,9171.44610,1.0,0.0
2019-11-07,9267.56,NaN,8687.2882,9604.2291,9191.21125,1.0,0.0
2019-11-08,8804.88,NaN,8658.0574,9591.4216,9208.23880,1.0,0.0
2019-11-09,8813.58,NaN,8630.6962,9575.5607,9224.44490,1.0,0.0


In [27]:
# Plot the moving averages
btc_df.hvplot(x="Date",y=['SMA50','SMA100','SMA200'])

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [28]:
# Visualize exit position relative to close price
exit = btc_df[btc_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='yellow',
	marker="v",
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
exit

# Visualize entry position relative to close price
entry = btc_df[btc_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='purple',
	marker = "^",
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
entry

# Visualize close price for the investment
security_close = btc_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
security_close

# Visualize moving averages
moving_avgs = btc_df[['SMA100', 'SMA200']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
moving_avgs

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [29]:
# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="BTC - SMA100, SMA200, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [30]:
# Initialize variable to hold previous day's trading price
# Set the initial value of the previous_price to 0
previous_price = 11475.3

# Initialize share size and accumulated shares
initial_capital = 10000
share_size = 100
accumulated_shares = 0





# Variable to track the previous purchase price
previous_purchase_price = 0

In [33]:
btc_df.loc["2019-11-01":"2019-12-17"]

,Close,Trade_Type,SMA50,SMA100,SMA200,Signal,Entry/Exit
Date,,,,,,,
2019-11-01,9261.10,NaN,8804.0676,9624.1851,9070.20360,1.0,0.0
2019-11-02,9324.72,NaN,8783.3510,9618.3139,9090.64940,1.0,0.0
2019-11-03,9235.35,NaN,8760.8970,9611.9644,9110.56645,1.0,0.0
2019-11-04,9412.61,NaN,8742.1950,9611.3137,9131.13755,1.0,0.0
2019-11-05,9342.53,NaN,8723.5098,9609.2104,9151.33115,1.0,0.0
2019-11-06,9360.88,NaN,8705.9020,9607.6277,9171.44610,1.0,0.0
2019-11-07,9267.56,NaN,8687.2882,9604.2291,9191.21125,1.0,0.0
2019-11-08,8804.88,NaN,8658.0574,9591.4216,9208.23880,1.0,0.0
2019-11-09,8813.58,NaN,8630.6962,9575.5607,9224.44490,1.0,0.0


# Trading Logic

In [41]:
# Loop through the Pandas DataFrame and initiate a trade each iteration
for index, row in btc_df.iterrows():
    if previous_purchase_price == 0: #& btc_df['Entry/Exit'] == float(1):
        btc_df.loc[index, 'Trade_Type'] = 'Buy'
        # Calculate cost of buy
        btc_df.loc[index, 'Cost/Proceeds'] = -(row['Close'] * share_size)
        # Add shares purchased to the number of accumulated shares
        accumulated_shares += share_size
    elif row['Close'] < previous_price:
        btc_df.loc[index, 'Trade_Type'] = 'Buy'
        # Calculate cost of buy
        btc_df.loc[index, 'Cost/Proceeds'] = -(row['Close'] * share_size)
        # Add shares purchased to the number of accumulated shares
        accumulated_shares += share_size
    elif row['Close'] > previous_price:
        btc_df.loc[index, 'Trade_Type'] = 'Sell'
        # Calculate proceeds of sell
        btc_df.loc[index, 'Cost/Proceeds'] = (row['Close'] * share_size)
        # Add shares purchased to the number of accumulated shares
        accumulated_shares -= share_size
    else:
        btc_df.loc[index, 'Trade_Type'] = 'Hold'
    
    # Update the previous_price to the current row's price
    previous_price = row["Close"]
    previous_purchase_price = row["Close"]
    
    # if the index is the last index of the DataFrame, sell
    # I THINK DELETE THIS IF PUT INTO PRACTICE
    if index == btc_df.index[-1]:
        btc_df.loc[index, "Trade_Type"] = "Sell"
        # Calculate proceeds of sell
        btc_df.loc[index, 'Cost/Proceeds'] = (row['Close'] * accumulated_shares)

In [43]:
display(accumulated_shares)
display(previous_purchase_price)
btc_df.head(10)

147600

39799.68

,Close,Trade_Type,SMA50,SMA100,SMA200,Signal,Entry/Exit,Cost/Proceeds
Date,,,,,,,,
2018-01-28,11786.30,Buy,NaN,NaN,NaN,0.0,NaN,-1178630.0
2018-01-29,11296.40,Buy,NaN,NaN,NaN,0.0,0.0,-1129640.0
2018-01-30,10106.30,Buy,NaN,NaN,NaN,0.0,0.0,-1010630.0
2018-01-31,10221.10,Sell,NaN,NaN,NaN,0.0,0.0,1022110.0
2018-02-01,9170.54,Buy,NaN,NaN,NaN,0.0,0.0,-917054.0
2018-02-02,8830.75,Buy,NaN,NaN,NaN,0.0,0.0,-883075.0
2018-02-03,9174.91,Sell,NaN,NaN,NaN,0.0,0.0,917491.0
2018-02-04,8277.01,Buy,NaN,NaN,NaN,0.0,0.0,-827701.0
2018-02-05,6955.27,Buy,NaN,NaN,NaN,0.0,0.0,-695527.0


# Profit/Loss Metrics

### Profit/Loss

In [16]:
# Calculate the total profit/loss for 100 share size orders
total_profit_loss = round(btc_df["Cost/Proceeds"].sum(), 2)
print(f"The total profit(-loss) is {total_profit_loss}")

The total profit(-loss) is 143494550.0


### Return on Investment

In [19]:
# Initialize the variable to hold the value of the invested capital
invested_capital = 0

# Calculate the invested capital by adding the cost of all buy trades
for index, row in btc_df.iterrows():
    if row["Trade_Type"] == "Buy":
        invested_capital = invested_capital + row["Cost/Proceeds"]


# Calculate the return on investment (ROI)
roi = round((total_profit_loss / -(invested_capital)) * 100, 2)

# Print the ROI
print(f"The trading algorithm resulted in a return on investment of {roi}%")

The trading algorithm resulted in a return on investment of 9.59%
